## Imports

In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
import torch.nn.functional as F
from lightning.pytorch import LightningModule, Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torchinfo import summary
from tqdm import tqdm
from transformers import XLMTokenizer, RobertaModel

from dataset import TextTrainDataset
from callback import GenerateCallback
from lstm import LstmTextGenerator

## Dataset

In [2]:
tokenizer = XLMTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")

In [6]:
dataset = TextTrainDataset('../../data/training', tokenizer, seq_length=20, padding=(3, 70), lowercase=True, tqdm=True, cache_path='.cache/dataset', cache_ignore=True, remove_special_chars=True, min_line_length=25)

100%|██████████| 1036/1036 [00:37<00:00, 27.90it/s]


In [6]:
len(dataset)

21570081

In [7]:
train_dataloader = DataLoader(
    dataset=dataset,
    batch_size=2048,
    shuffle=True,
    num_workers=0
)

## Model creation

In [3]:
lstm = LstmTextGenerator(
    # files
    train_dataset_path='../../data/training/',
    
    # architecture
    embedding_dim=300,
    lstm_layers=3,
    lstm_dropout=0.2,
    lstm_hidden_size=1024,
    dropout=0.2,
    bidirectional=True,
    
    # training
    lr=0.001,
    seq_length=20,
    padding=(3, 40),
    batch_size=512
)

In [ ]:
lstm = LstmTextGenerator.load_from_checkpoint('../lstm/logs/version_23/checkpoints/epoch=57-step=187000.ckpt')

In [7]:
summary(
    lstm,
    input_size=(512, 20),
    col_names=['input_size', 'output_size', 'num_params', 'params_percent'],
    dtypes=[torch.LongTensor],
    device='cpu'
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Param %
LstmTextGenerator                        [512, 20]                 [512, 50560]              --                             --
├─Embedding: 1-1                         [512, 20]                 [512, 20, 300]            15,168,000                  8.43%
├─LSTM: 1-2                              [512, 20, 300]            [512, 20, 2048]           61,227,008                 34.02%
├─Dropout: 1-3                           [512, 20, 2048]           [512, 20, 2048]           --                             --
├─Linear: 1-4                            [512, 2048]               [512, 50560]              103,597,440                57.56%
Total params: 179,992,448
Trainable params: 179,992,448
Non-trainable params: 0
Total mult-adds (G): 687.77
Input size (MB): 0.08
Forward/backward pass size (MB): 399.44
Params size (MB): 719.97
Estimated Total Size (MB): 1119.49

## Training

In [9]:
logger = TensorBoardLogger(
    save_dir='../..',
    name='logs'
)

generate_callback = GenerateCallback(
    'Pewnego dnia czerwony kapturek szedł przez las z koszyczkiem jedzenia do swojej babci, która mieszkała w lesie. Śledził go jednak zły wilk, który chciał zjeść dziewczynkę.',
    temperatures=[0.01, 0.1, 0.2, 0.3, 0.5, 0.7],
    length=200,
    interval=1000
)

checkpoint_callback = ModelCheckpoint(
    save_last=True,
    every_n_train_steps=1000,
)

trainer = Trainer(
    accelerator='cuda',
    max_epochs=-1,
    enable_progress_bar=True,
    logger = logger,
    callbacks=[generate_callback, checkpoint_callback],
    gradient_clip_val=0.4,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(lstm, train_dataloaders=train_dataloader, ckpt_path='../../logs/version_21/checkpoints/epoch=43-step=161906.ckpt')

You are using a CUDA device ('NVIDIA GeForce RTX 3080 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at ../../logs/version_21/checkpoints/epoch=43-step=161906.ckpt
/home/lklimkiewicz/stud/piat/Story-Generator/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:243: UserWarning: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': None, 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None}"].
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             |

Epoch 57:  56%|█████▌    | 1047/1876 [28:07<22:16,  1.61s/it, v_num=23]

/home/lklimkiewicz/stud/piat/Story-Generator/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## Testing

In [8]:
lstm.generate('Pewnego słonecznego dnia czerwony kapturek szedł do swojej babci z koszyczkiem. Kapturek był koloru', temperature=0.3)

'Pewnego słonecznego dnia czerwony kapturek szedł do swojej babci z koszyczkiem. Kapturek był koloru i wesoły, ale nie miał orzechów, więc ruszył z kopyta do łóżka, aby nie mógł się już doczekać, kiedy nagle usłyszano z daleka radosne okrzyki. a kiedy się obudziły, don kichot, widząc, że'

In [11]:
lstm.generate('Dawno, dawno temu był sobie kotek', temperature=0.2)

'Dawno, dawno temu był sobie kotek, a w nim mieszkała mała dziewczynka o imieniu córeczka. miała bardzo wyjątkowe buty, a dziewczynka wesoło bawiła się z nią bardzo z płatka. czuła się bardzo zmęczona, ale szczęśliwa nie wiedziała, co to znaczy i co się stało.'